In [1]:
import os
import glob
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.nn.functional import interpolate

In [2]:
import torch.nn.functional as F
import math


class DoubleConv(nn.Module):
    """ [(Conv2d) => (BN) => (ReLu)] * 2 """

    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding="same", stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, 3, padding="same", stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.double_conv(x)

class DownSample(nn.Module):
    """ MaxPool => DoubleConv """
    def __init__(self,in_channels,out_channels) -> None:
        super().__init__()
        self.down_sample = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels,out_channels)
        )
    def forward(self,x):
        x  = self.down_sample(x)
        return x



class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.conv(x)



class UpSample(nn.Module):
    def __init__(self,in_channels,out_channels,c:int) -> None:
        """ UpSample input tensor by a factor of `c`
                - the value of base 2 log c defines the number of upsample
                layers that will be applied
        """
        super().__init__()
        n = 0 if c == 0 else int(math.log(c,2))

        self.upsample = nn.ModuleList(
            [nn.ConvTranspose2d(in_channels,in_channels,2,2) for i in range(n)]
        )
        self.conv_3 = nn.Conv2d(in_channels,out_channels,3,padding="same",stride=1)

    def forward(self,x):
        for layer in self.upsample:
            x = layer(x)
        return self.conv_3(x)

class UpSample2(nn.Module):
    def __init__(self,in_channels,out_channels,c:int) -> None:
        """ UpSample input tensor by a factor of `c`
                - the value of base 2 log c defines the number of upsample
                layers that will be applied
        """
        super().__init__()
        n = 0 if c == 0 else int(math.log(c,2))
        #print(f'LOG OF C:  {int(math.log(c,2))}')

        self.upsample = nn.ModuleList(
            [nn.ConvTranspose2d(in_channels,in_channels,2,2) for i in range(n)]
        )
        self.conv_3 = nn.Conv2d(in_channels,out_channels,3,padding="same",stride=1)

    def forward(self,x):
        for layer in self.upsample:
            #print(f'BEFORE UPSAMPEL: {x.shape}')
            x = layer(x)
            #print(f'After Transpose2D: {x.shape}')
            x = self.conv_3(x)
            #print(f'After Conv2D: {x.shape}')
        return x#self.conv_3(x)

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels, n: int = 8) -> None:
        """
        Construct the J-net model.
        Args:
            in_channels: The number of color channels of the input image. 0:for binary 3: for RGB
            out_channels: The number of color channels of the input mask, corresponds to the number
                            of classes.Includes the background
            n: Channels size of the first CNN in the encoder layer. The bigger this value the bigger
                the number of parameters of the model. Defaults to n = 8, which is recommended by the
                authors of the paper.
        """
        super().__init__()
        # ------ Input convolution --------------
        self.in_conv = DoubleConv(in_channels, n)
        # -------- Encoder ----------------------
        self.down_1 = DownSample(n, 2 * n)
        self.down_2 = DownSample(2 * n, 4 * n)
        self.down_3 = DownSample(4 * n, 8 * n)
        self.down_4 = DownSample(8 * n, 16 * n)
        

        # -------- Upsampling ------------------ias=Tru
        self.up_1024_512 = UpSample(16 * n, 8 * n, 2)

        self.up_512_64 = UpSample(8 * n, n, 8)
        
        #self.up_512_64_more = UpSample(8 * n, n, 16)
        
        
        self.up_512_128 = UpSample(8 * n, 2 * n, 4)
        self.up_512_256 = UpSample(8 * n, 4 * n, 2)
        self.up_512_512 = UpSample(8 * n, 8 * n, 0)

        self.up_256_64 = UpSample(4 * n, n, 4)
        #self.up_256_64_more = UpSample(4 * n, n, 8)
        self.up_256_128 = UpSample(4 * n, 2 * n, 2)
        self.up_256_256 = UpSample(4 * n, 4 * n, 0)

        self.up_128_64 = UpSample(2 * n, n, 2)
        #self.up_128_64_more = UpSample(2 * n, n, 4)
        self.up_128_128 = UpSample(2 * n, 2 * n, 0)

        self.up_64_64 = UpSample(n, n, 0)
        m = int(n *0.5)
        
        self.up_64_32 =  UpSample2(out_channels, out_channels, 2)
        self.up_16_64 = UpSample(out_channels, out_channels, 2)
        
  
        self.up_skip_8 =  UpSample2(out_channels, out_channels, 8)
        self.up_skip_4 =  UpSample2(out_channels, out_channels, 4)
        
        
        self.up_concat =  UpSample2(2*out_channels, out_channels, 2)

        # ------ Decoder block ---------------
        self.dec_4 = DoubleConv(2 * 8 * n, 8 * n)
        self.dec_3 = DoubleConv(3 * 4 * n, 4 * n)
        self.dec_2 = DoubleConv(4 * 2 * n, 2 * n)
        self.dec_1 = DoubleConv(5 * n, n )
         # ------ Output convolution

        self.out_conv = OutConv(n, out_channels)
        
        

    def forward(self, x):
        x = self.in_conv(x)  # 64
        # ---- Encoder outputs
        x_enc_1 = self.down_1(x)  # 128
        x_enc_2 = self.down_2(x_enc_1)  # 256
        x_enc_3 = self.down_3(x_enc_2)  # 512
        x_enc_4 = self.down_4(x_enc_3)  # 1024

        # ------ decoder outputs
        x_up_1 = self.up_1024_512(x_enc_4)
        x_dec_4 = self.dec_4(torch.cat([x_up_1, self.up_512_512(x_enc_3)], dim=1))

        x_up_2 = self.up_512_256(x_dec_4)
        x_dec_3 = self.dec_3(torch.cat([x_up_2,
                                        self.up_512_256(x_enc_3),
                                        self.up_256_256(x_enc_2)
                                        ],
                                       dim=1))

        x_up_3 = self.up_256_128(x_dec_3)
        x_dec_2 = self.dec_2(torch.cat([
            x_up_3,
            self.up_512_128(x_enc_3),
            self.up_256_128(x_enc_2),
            self.up_128_128(x_enc_1)
        ], dim=1))

        x_up_4 = self.up_128_64(x_dec_2)
        x_dec_1 = self.dec_1(torch.cat([
            x_up_4,
            self.up_512_64(x_enc_3),
            self.up_256_64(x_enc_2),
            self.up_128_64(x_enc_1),
            self.up_64_64(x)
        ], dim=1))

        out1 =  self.out_conv(x_dec_1) # 16 / 32 / 80
        
        out2 = self.up_64_32(out1) # 32   / 64 / 160
        out3 = self.up_16_64(out2) # 64   / 128 / 320
        
        
        
        
        out1_skip_4 = self.up_skip_4(out1)      
        out1_skip_8 = self.up_skip_8(out1)  
        

        out4 = self.up_concat(torch.cat([out1_skip_4,out3],dim=1))
        out5 = self.up_concat(torch.cat([out1_skip_8,out4],dim=1))
        
       

        return out1, out2, out3, out4, out5
        
def save_net(ckpt_dir, net, optim, epoch):
    os.makedirs(ckpt_dir, exist_ok=True)
    path = os.path.join(ckpt_dir, f'model_epoch_{epoch}.pth')
    torch.save({'epoch': epoch, 'model': net.state_dict(), 'optim': optim.state_dict()}, path)

def load_net(ckpt_dir, net, optim):
    if not os.path.exists(ckpt_dir): return net, optim, 0
    files = sorted(glob.glob(os.path.join(ckpt_dir, '*.pth')))
    if not files: return net, optim, 0
    checkpoint = torch.load(files[-1])
    net.load_state_dict(checkpoint['model'])
    optim.load_state_dict(checkpoint['optim'])
    return net, optim, checkpoint['epoch'] + 1


In [3]:
import os
import glob
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# === Paths and Config ===
ROOT_DIR = "/kaggle/working"
DATA_DIR = os.path.join(ROOT_DIR, 'prostate_data')
CKPT_DIR = os.path.join(ROOT_DIR, 'checkpoints_jnet_16')
LOG_DIR = os.path.join(ROOT_DIR, 'logs')
IMGS_DIR = os.path.join(DATA_DIR, 'imgs_r')
LABELS_DIR = os.path.join(DATA_DIR, 'labels')

# === Create directories ===
for path in [
    CKPT_DIR, LOG_DIR, os.path.join(LOG_DIR, 'train'), os.path.join(LOG_DIR, 'val'),
    os.path.join(IMGS_DIR, 'train'), os.path.join(IMGS_DIR, 'val'), os.path.join(IMGS_DIR, 'test'),
    os.path.join(LABELS_DIR, 'train'), os.path.join(LABELS_DIR, 'val'), os.path.join(LABELS_DIR, 'test')
]:
    os.makedirs(path, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# === Config ===
class Config:
    LEARNING_RATE = 1e-3
    BATCH_SIZE = 64
    NUM_EPOCHS = 200
    L1_LAMBDA = 1e-5
    L2_LAMBDA = 1e-5
    IOU_LAMBDA = 1.0
    DICE_LAMBDA = 1.0

cfg = Config()

# === Dataset ===
class CustomDataset(Dataset):
    def __init__(self, imgs_dir, labels_dir, transform=None):
        self.imgs = sorted(glob.glob(os.path.join(imgs_dir, '*.jpg')))
        self.labels = sorted(glob.glob(os.path.join(labels_dir, '*.jpg')))
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        # Load RGB image
        img = cv2.imread(self.imgs[index], cv2.IMREAD_COLOR) / 255.0  # shape: (H, W, 3)
        
        # Load label as grayscale binary mask
        label = cv2.imread(self.labels[index], cv2.IMREAD_GRAYSCALE) / 255.0  # shape: (H, W)

        # Resize for deep supervision
        img_d = cv2.resize(img, (16, 16), interpolation=cv2.INTER_NEAREST)
        label_d = cv2.resize(label, (16, 16), interpolation=cv2.INTER_NEAREST)

        # Resize main
        img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_NEAREST)
        label = cv2.resize(label, (256, 256), interpolation=cv2.INTER_NEAREST)

        sample = {
            'img': img,
            'label': label[:, :, np.newaxis],  # add channel dim
            'img_d': img_d,
            'label_d': label_d[:, :, np.newaxis],
            'id': os.path.basename(self.imgs[index])[:-4]
        }

        if self.transform:
            sample = self.transform(sample)

        return sample

# === Transforms ===
class ToTensor:
    def __call__(self, data):
        for k in ['img', 'img_d']:
            data[k] = torch.tensor(data[k].transpose(2, 0, 1), dtype=torch.float32)
        for k in ['label', 'label_d']:
            data[k] = torch.tensor(data[k].transpose(2, 0, 1), dtype=torch.float32)
        return data

class RandomFlip:
    def __call__(self, data):
        if np.random.rand() > 0.5:
            for k in ['img', 'label', 'img_d', 'label_d']:
                data[k] = np.flip(data[k], axis=1).copy()
        if np.random.rand() > 0.5:
            for k in ['img', 'label', 'img_d', 'label_d']:
                data[k] = np.flip(data[k], axis=0).copy()
        return data

# === Compose Transforms ===
train_transform = transforms.Compose([
    RandomFlip(),
    ToTensor()
])

val_transform = transforms.Compose([
    ToTensor()
])

# === Loaders ===
train_set = CustomDataset(
    imgs_dir="/kaggle/input/kasvir-svg-train-val/train/images",
    labels_dir="/kaggle/input/kasvir-svg-train-val/train/masks",
    transform=train_transform
)

val_set = CustomDataset(
    imgs_dir="/kaggle/input/kasvir-svg-train-val/val/images",
    labels_dir="/kaggle/input/kasvir-svg-train-val/val/masks",
    transform=val_transform
)

train_loader = DataLoader(train_set, batch_size=cfg.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=cfg.BATCH_SIZE)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import os
import numpy as np

# === Dice Loss & Score ===
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum(dim=(1, 2, 3))
    dice = (2. * intersection + smooth) / (pred.sum(dim=(1, 2, 3)) + target.sum(dim=(1, 2, 3)) + smooth)
    return 1 - dice.mean()

def dice_score(pred, target, smooth=1e-6):
    pred = (torch.sigmoid(pred) > 0.5).float()
    intersection = (pred * target).sum(dim=(1, 2, 3))
    score = (2. * intersection + smooth) / (pred.sum(dim=(1, 2, 3)) + target.sum(dim=(1, 2, 3)) + smooth)
    return score.mean().item()

# === IoU Loss & Score ===
def iou_loss(pred, target, smooth=1e-6):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum(dim=(1, 2, 3))
    union = (pred + target - pred * target).sum(dim=(1, 2, 3))
    iou = (intersection + smooth) / (union + smooth)
    return 1 - iou.mean()

def iou_score(pred, target, smooth=1e-6):
    pred = (torch.sigmoid(pred) > 0.5).float()
    intersection = (pred * target).sum(dim=(1, 2, 3))
    union = (pred + target - pred * target).sum(dim=(1, 2, 3))
    return ((intersection + smooth) / (union + smooth)).mean().item()

# === Model, Loss, Optimizer ===
net = UNet(3, 1, 16).to(device)
loss_fn = nn.BCEWithLogitsLoss().to(device)
optim = torch.optim.Adam(net.parameters(), lr=cfg.LEARNING_RATE, weight_decay=cfg.L2_LAMBDA)

train_writer = SummaryWriter(os.path.join(LOG_DIR, 'train'))
val_writer = SummaryWriter(os.path.join(LOG_DIR, 'val'))
start_epoch = 0

# === Training Loop ===
for epoch in range(start_epoch, cfg.NUM_EPOCHS):
    net.train()
    train_losses, bce_losses, dice_losses, iou_losses, l1_losses = [], [], [], [], []
    dice_scores, iou_scores = [], []

    for batch in train_loader:
        img = batch['img_d'].to(device)
        label = batch['label'].to(device)
        label_d = batch['label_d'].to(device)

        output1, output2, output3, output4, output5 = net(img)

        resized = [
            F.interpolate(label, size=(32, 32), mode='bilinear', align_corners=False),
            F.interpolate(label, size=(64, 64), mode='bilinear', align_corners=False),
            F.interpolate(label, size=(128, 128), mode='bilinear', align_corners=False)
        ]

        # === Losses ===
        bce_loss = (
            loss_fn(output1, label_d) +
            loss_fn(output2, resized[0]) +
            loss_fn(output3, resized[1]) +
            loss_fn(output4, resized[2]) +
            loss_fn(output5, label)
        )

        dice = dice_loss(output5, label)
        iou = iou_loss(output5, label)

        # L1 Regularization
        l1_reg = torch.tensor(0., requires_grad=True).to(device)
        for param in net.parameters():
            l1_reg = l1_reg + torch.norm(param, 1)

        # Total Loss
        loss = bce_loss + cfg.DICE_LAMBDA * dice + cfg.IOU_LAMBDA * iou + cfg.L1_LAMBDA * l1_reg

        # Backward
        optim.zero_grad()
        loss.backward()
        optim.step()

        # === Logging individual losses ===
        train_losses.append(loss.item())
        bce_losses.append(bce_loss.item())
        dice_losses.append(dice.item())
        iou_losses.append(iou.item())
        l1_losses.append(l1_reg.item())
        dice_scores.append(1 - dice.item())
        iou_scores.append(1 - iou.item())

        print(f'[Train] Epoch {epoch}/{cfg.NUM_EPOCHS} | Total: {loss.item():.4f} | BCE: {bce_loss.item():.4f} | Dice: {1 - dice.item():.4f} | IoU: {1 - iou.item():.4f} | L1: {l1_reg.item():.4f}')

    # === TensorBoard Logging ===
    train_writer.add_scalar('train/total_loss', np.mean(train_losses), epoch)
    train_writer.add_scalar('train/bce_loss', np.mean(bce_losses), epoch)
    train_writer.add_scalar('train/dice_loss', np.mean(dice_losses), epoch)
    train_writer.add_scalar('train/iou_loss', np.mean(iou_losses), epoch)
    train_writer.add_scalar('train/l1_loss', np.mean(l1_losses), epoch)
    train_writer.add_scalar('train/dice_score', np.mean(dice_scores), epoch)
    train_writer.add_scalar('train/iou_score', np.mean(iou_scores), epoch)

    # === Validation ===
    net.eval()
    val_losses, val_dice_scores, val_iou_scores = [], [], []

    with torch.no_grad():
        for batch in val_loader:
            img = batch['img_d'].to(device)
            label = batch['label'].to(device)
            id_list = batch['id']

            outputs = net(img)
            output_final = outputs[-1]
            val_loss = loss_fn(output_final, label)

            val_losses.append(val_loss.item())
            val_dice_scores.append(dice_score(output_final, label))
            val_iou_scores.append(iou_score(output_final, label))

            if epoch == cfg.NUM_EPOCHS - 1:
                os.makedirs('./pred_out/', exist_ok=True)
                for i, id in enumerate(id_list):
                    np.save(f'./pred_out/{id}.npy', output_final[i].cpu().numpy())

            print(f'[Val] Epoch {epoch} | BCE Loss: {val_loss.item():.4f} | Dice: {val_dice_scores[-1]:.4f} | IoU: {val_iou_scores[-1]:.4f}')

    val_writer.add_scalar('val/bce_loss', np.mean(val_losses), epoch)
    val_writer.add_scalar('val/dice_score', np.mean(val_dice_scores), epoch)
    val_writer.add_scalar('val/iou_score', np.mean(val_iou_scores), epoch)

    # === Summary ===
    print(f'==> Epoch {epoch}')
    print(f'    Train Total Loss : {np.mean(train_losses):.4f} | BCE: {np.mean(bce_losses):.4f} | Dice score: {np.mean(dice_scores):.4f} | IoU score: {np.mean(iou_scores):.4f} | L1: {np.mean(l1_losses):.4f}')
    print(f'    Val   BCE Loss   : {np.mean(val_losses):.4f} | Dice score: {np.mean(val_dice_scores):.4f} | IoU score : {np.mean(val_iou_scores):.4f}')

    if epoch % 5 == 0 or epoch in {1, 4}:
        save_net(CKPT_DIR, net, optim, epoch)

train_writer.close()
val_writer.close()


[Train] Epoch 0/200 | Total: 5.5031 | BCE: 3.3268 | Dice: 0.2021 | IoU: 0.1178 | L1: 49625.6133
[Train] Epoch 0/200 | Total: 5.4412 | BCE: 3.2755 | Dice: 0.1952 | IoU: 0.1132 | L1: 47415.6133
[Train] Epoch 0/200 | Total: 5.3661 | BCE: 3.2536 | Dice: 0.2140 | IoU: 0.1250 | L1: 45153.4102
[Train] Epoch 0/200 | Total: 5.3387 | BCE: 3.2497 | Dice: 0.2148 | IoU: 0.1252 | L1: 42897.1484
[Train] Epoch 0/200 | Total: 5.3176 | BCE: 3.2197 | Dice: 0.1956 | IoU: 0.1134 | L1: 40697.4727
[Train] Epoch 0/200 | Total: 5.2735 | BCE: 3.2104 | Dice: 0.2036 | IoU: 0.1188 | L1: 38556.1172
[Train] Epoch 0/200 | Total: 5.3065 | BCE: 3.1900 | Dice: 0.1590 | IoU: 0.0894 | L1: 36485.8164
[Train] Epoch 0/200 | Total: 5.1683 | BCE: 3.2008 | Dice: 0.2364 | IoU: 0.1410 | L1: 34495.9180
[Train] Epoch 0/200 | Total: 5.1580 | BCE: 3.1920 | Dice: 0.2260 | IoU: 0.1339 | L1: 32591.1035
[Train] Epoch 0/200 | Total: 5.1687 | BCE: 3.1832 | Dice: 0.2031 | IoU: 0.1191 | L1: 30778.4180
[Train] Epoch 0/200 | Total: 5.1324 | BC